# 🛳️ **Titanic - Machine Learning from Disaster**   
This project aims to predict which passengers survived the Titanic shipwreck using historical data.  
It demonstrates an end-to-end machine learning workflow including data cleaning, feature engineering, model training, and evaluation.


We load the Titanic dataset provided by Kaggle and begin understanding the structure, types of variables, and initial issues like missing values.  

Key columns to explore:  
- Categorical: Sex, Embarked, Pclass, Cabin  
- Numerical: Age, Fare, SibSp, Parch


In [ ]:
# Importing Relevant Libraries 

# Data Preparation

import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

# Modelling & Feature Selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

# Evaluation & Metrics
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_score, recall_score, f1_score
)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
overall = pd.concat([train_data,test_data])
overall.isnull().sum()

# 📊 2. Exploratory Data Analysis (EDA)
Before modeling, we explore:
- Missing value patterns
- Survival rates across key groups (e.g., Sex, Pclass)
- Age and Fare distributions
- Correlation among features

Insights will guide our feature engineering.


In [ ]:
plt.rc('font',size=14)
plt.rc('axes',labelsize=14, titlesize=14)
plt.rc('legend',fontsize=14)
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)

train_data.hist(bins=50,figsize=(12,8))
plt.show()

Checking Count of Embarked values to estimate for filling null values

In [ ]:
sns.countplot(data=overall,x='Embarked')
plt.title("Passenger count by Embarked Port")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

#Sex vs Survived
sns.barplot(data=overall, x='Sex', y='Survived', ax=axes[0], palette='pastel')
axes[0].set_title('Survival Rate by Sex')
axes[0].set_ylabel('Survival Rate')

#Pclass vs Survived
sns.barplot(data=overall, x='Pclass', y='Survived', ax=axes[1], palette='muted')
axes[1].set_title('Survival Rate by Pclass')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()


Filling null values with M and checking Cabin values distribution

In [ ]:
overall['Cabin'] = overall['Cabin'].apply(lambda x : str(x)[0] if pd.notnull(x) else 'M')

sns.countplot(data=overall,x='Cabin')

# 🧱 3. Data Cleaning
Steps performed:

There are null values in Age, Cabin, Embarked and Fare.  
1. Null Age is filled by median.  
2. Embarked is filled by 'S' as only 2 values is missing and it is the mode.  
3. Cabins majority value is null. To handle this I am replacing null values with 'Missing'.  
4. Fare null value is filled with mean as only 1 value is missing. Also, there is long tail (skewness) in Fare value hence I am taking log1p as Fare values are small.

In [ ]:
train_data['Fare_to_Class'] = train_data['Fare'] / train_data['Pclass']

In [ ]:

corr_matrix = train_data.corr(numeric_only=True)
corr_matrix['Survived'].sort_values(ascending=False)

There is very less correlation between Passenger ID and Survival and it makes sense as it is just sequence of numbers.  
Fare_to_class have a better correlation than Fare.

# 🧠 4. Feature Engineering
Defining a function to preprocess all the data.
1. Extracted 'Title' from Name. Divided this into [Officers - Working on Deck, Nobles - High Worth People, Married Female - Usually preferred during Evacuation, Common Male and Unmarried Female.
2. Extracted Ticket Number and Ticket Prefix from Ticket.
3. Created Age Band each of 16 years.
4. Created Total Family members count inclusing self (SplSb + Parch + 1)
5. Also added indicator column for passenger: is alone or not (IsAlone)
6. Dropped less contributing columns

In [ ]:
def preprocess(df):
    
    def ticket_split(x):
        return x.split(" ")[-1]

    def ticket_pref(x):
        pref = x.split(" ")
        if len(pref) == 1:
            return "NONE"
        return '_'.join(pref[0:-1])

    def Title(df):
        df['Title'] = df['Name'].str.extract(r'^[^,]+,\s*(.*?)\.')[0]
        df['Title'] = df['Title'].replace(['Ms', 'Mlle', 'Miss'],'Unmarried Female')
        df['Title'] = df['Title'].replace(['Mr','Master'],'Common Male')
        df['Title'] = df['Title'].replace(['Mrs', 'Mme'],'Married Female')
        df['Title'] = df['Title'].replace(['Col', 'Capt', 'Major', 'Rev', 'Dr'],'Officers')
        df['Title'] = df['Title'].replace(['Don', 'Jonkheer', 'the Countess','Dona', 'Sir', 'Lady'],'Nobles')
        return df

    df = Title(df)

    df['Title'] = df['Title'].str[0]
    
    def name(x):
        return " ".join(v.strip(",()[].\"'") for v in x.split(" "))

    df['Name'] = df['Name'].apply(name)
    df['Ticket_Number'] = df['Ticket'].apply(ticket_split)
    df['Ticket_Prefix'] = df['Ticket'].apply(ticket_pref)

    df['Age'] = df['Age'].fillna(value=df['Age'].median())
    df['Embarked'] = df['Embarked'].fillna('S')
    
    bins = [0,16,32,48,64,float('inf')]
    labels = [0,1,2,3,4]
    df['Age_Band'] = pd.cut(df['Age'],bins=bins,labels=labels,right=True)

    df['Cabin'] = df['Cabin'].fillna('Missing')
    df['Cabin'] = df['Cabin'].str[0]

    df['Fare'] = df['Fare'].fillna(value=df['Fare'].mean())
    df['Fare'] = np.log1p(df['Fare'])

    df['Fare_to_Class'] = df['Fare'] / df['Pclass']
    
    df['Fam_Count'] = df['SibSp']+df['Parch']+1

    df['IsAlone'] = df['Fam_Count'].apply(lambda x: 1 if x==1 else 0)
    
    df = df.drop(['PassengerId','Name','Ticket','Age','Pclass','Fare'],axis=1)
    
    return df
    


In [ ]:
preprocessed_train_df = preprocess(train_data)
preprocessed_test_df = preprocess(test_data)
preprocessed_train_df.info()

In [ ]:
cat = preprocessed_train_df.select_dtypes("object").columns
num = ['Pclass','SibSp','Parch','Fare','Age_band','Fam_Count','IsAlone']

In [ ]:
y_train = preprocessed_train_df['Survived']
X_train = preprocessed_train_df.drop(['Survived'],axis=1)
X_test = preprocessed_test_df

Creating Train and Test Split for validation

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

I am using column Transformer to tranform Categorical columns using OneHotEncoder and Numericals using StandardScaler.  
Adding SelectFromModel as it selects the features with decent feature_importances_  
Using RandomForestClassifier for this Classification problem (It works well as it reduces overfitting compared to single decision tree, provide feature_importances_ to study impact of features on label)

In [ ]:
cat_encoder = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
rf_clf = RandomForestClassifier(random_state=42)
feature_selector = SelectFromModel(RandomForestClassifier(random_state=42))
col_transform = make_column_transformer((cat_encoder,cat),remainder = StandardScaler())

In [ ]:
pipeline = Pipeline([('col',col_transform),('clf',rf_clf)])

Training on Processed Data

In [ ]:
pipeline.fit(X_train,y_train)

I used GridSearchCV to tune hyperparameters like n_estimators, max_depth, min_sample_split, max_features).

In [ ]:
param_grid = {'clf__n_estimators':[64, 128, 200],
             'clf__max_features':[2, 4, 'sqrt'],
             'clf__max_depth':[2, 3, 4],
             'clf__min_samples_split':[2, 3,4],
             'clf__min_samples_leaf': [2,3,4],
             'clf__max_samples':[0.8, 1]}

In [ ]:
RF_grid = GridSearchCV(estimator=pipeline,param_grid=param_grid,scoring='recall',
                       n_jobs=-1,verbose=1)

In [ ]:
RF_grid.fit(X_train,y_train)

In [ ]:
RF_grid.best_params_

In [ ]:
predict_val = RF_grid.predict(X_val)

In [ ]:
features = pd.DataFrame(pipeline['clf'].feature_importances_ , columns=['Imp'] , index = pipeline['col'].get_feature_names_out())
features.sort_values(by='Imp',ascending=False).head()

Checked Precision, Recall and F1 score on validation set.

In [ ]:
precision = precision_score(y_val,predict_val)
print(f"Precision : {precision:.4f}")

recall = recall_score(y_val,predict_val)
print(f"Recall : {recall:.4f}")

f1 = f1_score(y_val,predict_val)
print(f"F1 : {f1:.4f}")

In [ ]:
cm = confusion_matrix(y_val,predict_val)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

The confusion matrix reveals how well the model is distinguishing between survivors and non-survivors. Out of the total, 97 survivors were correctly identified (True Positives), while only 8 were missed (False Negatives). However, 36 non-survivors were mistakenly classified as survivors (False Positives), indicating a slight bias toward predicting survival. The model also correctly flagged 38 individuals as non-survivors (True Negatives). Overall, the model favors recall for survival, potentially prioritizing fewer missed survivors at the cost of more false alarms.

To improve the model’s ability to identify survivors, exploring strategies such as tuning classification thresholds, experimenting with different models or ensemble methods may help distinguish survival patterns more accurately. Priority will be placed on minimizing false negatives, as missing actual survivors can be more critical in this context.

In [ ]:
scores = cross_val_score(RF_grid.best_estimator_, X_train, y_train,cv=3)
print("Cross-validation scores (3-fold) : ",scores)
print("Average CV accuracy = ",round(scores.mean() ,4))

In [ ]:
predictions = RF_grid.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

# 📈 5. Model Evaluation
Trained the model on preprocessed training data.
- Achieved validation accuracy of ~80.4%
- Submitted results to Kaggle and achieved public leaderboard score of 0.78708 (Top - 11%)


# ✅ 6. Final Summary
In this project, I built a machine learning model to predict passenger survival on the Titanic dataset. After exploring the data and engineering new features like `Title`, `FamilySize`, and `IsAlone`, I trained a Random Forest classifier to perform the prediction task.
Key steps:
- Performed basic data cleaning and handled missing values (e.g., Age, Embarked).
- Created meaningful features from existing columns to improve prediction power.
- Trained a Random Forest model using GridSearchCV for hyperparameter tuning to balance bias and variance.
- Achieved a **public leaderboard score of 0.78708**, placing the model above baseline solutions.
   
Next steps to improve the model in future:
- Add cross-validation (e.g., Stratified K-Fold) to get a more reliable performance estimate.
- Try other models like XGBoost, Gradient Boosting, or stacking ensembles.
- Tune hyperparameters using GridSearchCV or RandomizedSearchCV.  


This project demonstrates my ability to apply the full ML pipeline : from data preprocessing and feature engineering to modeling and evaluation : on real-world tabular data.

Please consider upvoting if you found value from this Notebook!!